In [21]:
import pickle
import os

In [22]:
# ディレクトリのパス
directory_path = "./results/"

# ディレクトリ内のファイルをリストとして取得
file_list = os.listdir(directory_path)

# ファイルリストを表示
for i,file in enumerate(file_list):
    print(i,file)

0 blp_0902.txt
1 blp_0904.txt
2 blp_0904_2015_slsqp.txt
3 logit_blp_len3.pkl
4 MNL_len3_2023-09-05_21-06-56.pkl
5 MNL_len4_2023-09-05_21-04-58.pkl
6 RCL_len4_2023-09-05_21-02-48.pkl


In [23]:
file_name=file_list[5]
with open(f"./results/{file_name}", "rb") as f:
    result = pickle.load(f)

In [24]:
# problem and rc_types
result.problem, result.problem.rc_types

(Dimensions:
  T     N     K1    MD    ED 
 ---  -----  ----  ----  ----
 344  29653   4     8     2  
 
 Formulations:
      Column Indices:          0      1         2         3  
 --------------------------  ------  ----  -----------  -----
 X1: Linear Characteristics  prices  oral  in_hospital  後発品区分
 ============================================================,
 [])

In [25]:
# constraints
result.sigma_bounds,result.pi_bounds

((array([], shape=(0, 0), dtype=float64),
  array([], shape=(0, 0), dtype=float64)),
 (array([], shape=(0, 0), dtype=float64),
  array([], shape=(0, 0), dtype=float64)))

In [26]:
# result
result

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -------------  -------  ----------------  -----------------
 2    +5.926081E+00     0      +2.982098E+18      +5.480772E+07  

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
    prices            oral          in_hospital         後発品区分     
---------------  ---------------  ---------------  ---------------
 -1.509749E-03    +1.522522E+00    +4.543447E-01    -2.449966E+00 
(+1.578384E-03)  (+9.525010E-02)  (+4.288114E-02)  (+7.768697E-01)

In [27]:
# time in minutes
result.cumulative_total_time/60

0.0006142377853393555